# Метод merge

На практике источники данных редко ограничиваются одной таблицей.  Что делать, если надо объединить несколько? 

Если все ваши данные сложены в одну хорошо настроенную базу данных, то объединение этих таблиц сводится к SQL JOIN командам. Но, предположим, такой работы еще никто не проделал, и вы, наверняка, даже не знаете, что такое SQL JOIN, тогда для данных, доступных к обработке в Pandas, есть метод merge. Он позволяет аналогичным образом объединять датафреймы, используя для объединения один или несколько общих столбцов таблиц.

В этом модуле мы продолжим использовать знакомый нам датасет от Grouplens. Предположим, вам поручили определить самые популярные жанры среди кинолюбителей, но жанры хранятся в другом файле. 

Помимо файла ratings.csv ,возьмем файл с соответствием movieId и названием фильма с перечнем его жанров movies.csv. Нам необходимо объединить эти файлы и определить самые популярные жанры. Поехали!

In [2]:
import pandas as pd
import numpy as np

In [14]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.shape

(9125, 3)

In [15]:
len(movies)

9125

Посчитайте какое количество фильмов в датафрейме movies относится к жанру Fantasy? Т. е. найдите количество строк, у которых в столбце genres упоминается слово 'Fantasy'. Напомним, что для такого фильтра можно использовать метод str.contains

In [17]:
len(movies[ movies['genres'].str.contains('Fantasy') ])

654

In [20]:
# Узнаем, сколько фильмов в датафрейме movies относится к жанру Fantasy
movies['genres'].str.contains('Fantasy').sum()

654

ОБЪЕДИНЯЕМ СО СТАТИСТИКОЙ РЕЙТИНГОВ

У датафреймов ratings и movies есть общий столбец movieId. А это значит, что мы можем объединить эти датафреймы в одну таблицу. Используем метод merge:

Основные параметры метода merge:

how — при значении left берем все значения из ratings и ищем их соответствия в movies. Если нет совпадений, то ставим нулевое значение. При этом все значения из ratings сохраняются. Другие варианты: right, inner (оставляем только те movieId, которые есть в обоих датафреймах), outer (объединение всех вариантов movieId в датафреймах).
on — по какому столбцу происходит объединение. Для объединения по нескольким столбцам используйте on = ['col1', 'col2'] или left_on и right_on.

In [18]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [19]:
#После объединения датафреймов лучше проверять, не возникли ли дубликаты. О возможных проблемах метода merge мы поговорим 
#в следующем блоке. Сейчас проверим, совпадает ли число строк объединенного датафрейма с исходным:

len(ratings) == len(joined)

True

### УПРАЖНЕНИЕ

Вам даны выгрузки статистики рекламных кампаний из Яндекс.Директа (файл direct_stats.tsv) и заказов на сайте из CRM (файл crm_stats.tsv). В файле со статистикой Яндекс.Директа приведены данные по открученной рекламе: для каждой кампании (столбец campaign) есть показы объявлений, клики и расходы. В файле с  выгрузкой CRM - для этих же кампаний сумма заказов на сайте (столбец orders).

Вам необходимо совместить эти выгрузки по дате и названию кампании. И посчитать стоимость заказа для каждой кампании за каждый день. Т. е. отношение расходов (столбец cost) к количеству заказов (столбец orders).

Какое значение стоимости заказа получится к кампании landings_promo за 1 января 2018? Ответ округлите до ближайшего целого числа. Пример ответа: 698

In [27]:
crm = pd.read_csv('crm_stats.tsv', sep='\t')
direct = pd.read_csv('direct_stats.tsv', sep='\t')
direct.head()

,date,campaign,views,clicks,cost
0,2018-01-01,landings_promo,38120423,49557,1139801
1,2018-01-01,homepage_partner_1,5729483,12605,189073
2,2018-01-01,homepage_partner_2,4412029,9265,176040
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555
4,2018-01-02,landings_promo,40873806,61311,1471457


In [23]:
crm.head()

,date,campaign,orders
0,2018-01-01,landings_promo,1487
1,2018-01-01,homepage_partner_1,386
2,2018-01-01,homepage_partner_2,315
3,2018-01-01,socdem_w_25-34_vip_test,85
4,2018-01-02,landings_promo,1605


In [25]:
print(len(crm))
print(len(direct))

28
28


In [29]:
# Совмещаем датафрейме по дате и названию компани1
join = crm.merge(direct, on=['date', 'campaign'])
join.head()

,date,campaign,orders,views,clicks,cost
0,2018-01-01,landings_promo,1487,38120423,49557,1139801
1,2018-01-01,homepage_partner_1,386,5729483,12605,189073
2,2018-01-01,homepage_partner_2,315,4412029,9265,176040
3,2018-01-01,socdem_w_25-34_vip_test,85,913823,2559,89555
4,2018-01-02,landings_promo,1605,40873806,61311,1471457


In [30]:
print(len(join))

28


In [34]:
# Считаем стоимость заказа
join['order_cost'] = join['cost'] / join['orders']
join.head()

,date,campaign,orders,views,clicks,cost,order_cost
0,2018-01-01,landings_promo,1487,38120423,49557,1139801,766.510424
1,2018-01-01,homepage_partner_1,386,5729483,12605,189073,489.826425
2,2018-01-01,homepage_partner_2,315,4412029,9265,176040,558.857143
3,2018-01-01,socdem_w_25-34_vip_test,85,913823,2559,89555,1053.588235
4,2018-01-02,landings_promo,1605,40873806,61311,1471457,916.795639


In [32]:
# Находим значение этого столбца для landings_promo за 1 января 2018
round(join[(join['campaign'] == 'landings_promo') & (join['date'] == '2018-01-01')]['order_cost'])

0    767.0
Name: order_cost, dtype: float64

## СЧИТАЕМ РЕЙТИНГИ ЖАНРОВ


Итак, теперь для каждой строки датафрейма joined нам нужно выделить рейтинги жанров из списка genres. Напомним список жанров, которые нам интересны:

In [ ]:
genres = ['Drama', 'Action', 'Thriller']